Runfile for CellPose segmentation. On top of saving the images, it also saves a .txt file containing the parameters chosen for the segmentation. I implemented it this way because I tried various different segmentations and this made it easier to keep track of what I did. The `notes` variable defined three cells down is just meant for some explanation of what you did that was specific to this segmentation. Can just be an empty string though. **Make sure to define a `segmentation_version` (name) which hasn't been used yet because otherwise the files in that folder will be overwritten.**

In [1]:
from cellpose import models
from skimage import io
from pathlib import Path
import string
import numpy as np
from skimage import img_as_uint
from skimage.filters import threshold_otsu
from datetime import datetime

In [2]:
def create_well_ids(nrows, ncols, n_wells): 
    row_names = list(string.ascii_uppercase)[:nrows]
    well_names = ["{}{:02d}".format(row_name, col_name) for row_name in row_names for col_name in range(1,ncols+1)]
    return well_names[:n_wells]

In [7]:
segmentation_version = "V2"  # Make sure to pick a name that doesn't exist yet
nucleus_chan = 3 # nucleus stain channel number
experiment_name = "plate14"
# For txt output file
notes = "Otsu threshold on Hoechst prior to segmentation"

# Path to images
img_dir = Path(r"/links/groups/treutlein/DATA/imaging/PW/4i/plate14_aligned_UZH")
# Output path for segmentation masks
out_dir = Path(r"/links/groups/treutlein/DATA/imaging/PW/4i/plate14_aligned_nuclei_{}".format(segmentation_version))
if out_dir.is_dir():
    print(5*"\nOUTPUT FOLDER ALREADY EXISTS, PICK A DIFFERENT NAME ('segmentation_version')")

ref_cycle = "cycle1"  # cycle that will be used to create segmentation masks
# Exceptions where another cycle should be used
ref_exceptions = {
    "G01": "cycle2",
    "G02": "cycle2"
}

# Create list of well names
# Specify dimensions of plate (e.g nrows=16 and ncols=24 for 384 well plate)
nrows = 7
ncols = 8
wells = create_well_ids(nrows=nrows, ncols=ncols, n_wells=50)


OUTPUT FOLDER ALREADY EXISTS, PICK A DIFFERENT NAME ('segmentation_version')
OUTPUT FOLDER ALREADY EXISTS, PICK A DIFFERENT NAME ('segmentation_version')
OUTPUT FOLDER ALREADY EXISTS, PICK A DIFFERENT NAME ('segmentation_version')
OUTPUT FOLDER ALREADY EXISTS, PICK A DIFFERENT NAME ('segmentation_version')
OUTPUT FOLDER ALREADY EXISTS, PICK A DIFFERENT NAME ('segmentation_version')


In [47]:
imgs = []
out_names = []
# Load images
for well in wells:
    # Define reference cycle
    if well in ref_exceptions:
        cycle = ref_exceptions[well]
    else:
        cycle = ref_cycle
    # Define image name, load image
    img_name = "{}_{}_C{:02d}.png".format(experiment_name, well, nucleus_chan)
    try:
        print(well, cycle)
        # Load image
        img = io.imread(str(img_dir/cycle/img_name))
        # Define name of segmentation mask file for this particular image
        out_name = "{}_{}_mask.png".format(experiment_name, well)
        # Store both the image and the name for its segmentation mask
        out_names.append(out_name)
        imgs.append(img)
    except FileNotFoundError:
        print("{} not found".format(well))

A01 cycle1
A02 cycle1
A03 cycle1
A04 cycle1
A05 cycle1
A06 cycle1
A07 cycle1
A08 cycle1
B01 cycle1
B02 cycle1
B03 cycle1
B04 cycle1
B05 cycle1
B06 cycle1
B07 cycle1
B08 cycle1
C01 cycle1
C02 cycle1
C03 cycle1
C04 cycle1
C05 cycle1
C06 cycle1
C07 cycle1
C08 cycle1
D01 cycle1
D02 cycle1
D03 cycle1
D04 cycle1
D05 cycle1
D06 cycle1
D07 cycle1
D08 cycle1
E01 cycle1
E02 cycle1
E03 cycle1
E04 cycle1
E05 cycle1
E06 cycle1
E07 cycle1
E08 cycle1
F01 cycle1
F02 cycle1
F03 cycle1
F04 cycle1
F05 cycle1
F06 cycle1
F07 cycle1
F08 cycle1
G01 cycle2
G02 cycle2


In [48]:
# Threshold
for img in imgs:
    thr = threshold_otsu(img)
    img[img < thr] = 0

In [ ]:
## Define cellpose model and parameters
model_type = "cyto"
model = models.Cellpose(gpu=False, model_type=model_type)
diam = 35
flow_thr = 0.8
cellprob_thr = 0
channels = [0,0]  # [0,0] for grayscale

start_time = datetime.now()
masks, flows, styles, diams = model.eval(imgs, diameter=diam, channels=channels, do_3D=False,
                                         flow_threshold=flow_thr, cellprob_threshold=cellprob_thr)

>>>> using CPU
cellpose_residual_on_style_on_concatenation_off
processing 50 image(s)


 22%|██▏       | 11/50 [5:32:38<17:43:25, 1636.04s/it]

In [ ]:
# Create output directory if it doesn't exist yet
if not Path(out_dir).is_dir():
    out_dir.mkdir(parents=True, exist_ok=True)

# Save segmentation masks
for i in range(len(masks)):
    mask = img_as_uint(masks[i])  # convert to uint16
    filename = out_names[i]
    io.imsave(out_dir/filename, mask)

end_time = datetime.now()

In [ ]:
# Write txt file to note down parameters
with open(Path(out_dir/"segmentation_params.txt"), "w") as file:
    file.write("Start:\t {}\n".format(start_time.strftime("%d.%m.%Y - %H:%M:%S")))
    file.write("Finish:\t {}\n\n".format(end_time.strftime("%d.%m.%Y - %H:%M:%S")))
    file.write("model_type: {}\n".format(model_type))
    file.write("diameter: {}\n".format(diam))
    file.write("flow_threshold: {}\n".format(flow_thr))
    file.write("cellprob_threshold: {}\n".format(cellprob_thr))
    file.write("channels: {}\n\n".format(channels))
    file.write("Notes:\n  {}".format(notes))

--- 
## Checking segmentation

In [58]:
%matplotlib widget

from skimage import io
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity
import numpy as np

In [59]:
def rescale_image(img, min_quant=0, max_quant=0.99): 
    img = img * 1.0 # turn type to float before rescaling
    min_val = np.quantile(img, min_quant)
    max_val = np.quantile(img, max_quant)
    img = rescale_intensity(img, in_range=(min_val, max_val))
    return img

def colour_nuclei(nuclei):
    coloured = np.zeros((nuclei.shape[0], nuclei.shape[1], 3), np.uint8)
    for n in range(nuclei.max()):
        pixels = (nuclei==n+1)
        coloured[pixels, :] = np.random.randint(1,255,3)
    return coloured

In [184]:
nuclei_dir = Path(r"/links/groups/treutlein/DATA/imaging/PW/4i/plate14_aligned_nuclei_V1")
img_dir = Path(r"/links/groups/treutlein/DATA/imaging/PW/4i/plate14_aligned_UZH")

experiment_name = "plate14"
well = "A04"
cycle = "cycle1"
nucleus_chan = 3

# Load images
img_file = "{}_{}_C{:02d}.png".format(experiment_name, well, nucleus_chan)
img = io.imread(str(img_dir/cycle/img_file))
nuc_file = "{}_{}_mask.png".format(experiment_name, well)
nuclei = io.imread(str(nuclei_dir/nuc_file))

Note: Colouring the nuclei in different colours takes a really long time for the whole image. It makes sense to first just look at the Hoechst image, look for an interesting region and then only showing the segmentation for that region to save time.

In [185]:
plt.close("all")
fig = plt.figure(figsize=(7,7))
plt.imshow(rescale_image(img), cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [186]:
# Define region of interest
y_min = 800
y_max = 6700
x_min = 2500
x_max = img.shape[0]

# Get coloured version of nuclei segmentation
coloured_nuclei = colour_nuclei(nuclei[y_min:y_max, x_min:x_max])
# Add alpha channel to make background transparent
alpha = np.all(coloured_nuclei != 0, axis=2) * 255
nuclei_rgba = np.dstack((coloured_nuclei, alpha)).astype(np.uint8)

In [187]:
# Dont mask the images directly
img_masked = img[y_min:y_max, x_min:x_max].copy()
nuc_masked = nuclei_rgba.copy()

In [180]:
# Potentially mask out areas
x_left = 4500
x_right = 4800
y_top = 4000
y_bottom = 4300

img_masked[y_top:y_bottom, x_left:x_right] = 0
nuc_masked[y_top:y_bottom, x_left:x_right] = 0

In [188]:
# Plot Hoechst next to segmentation
plt.close("all")
fig, ax = plt.subplots(1,2, figsize=(10,6), sharex=True, sharey=True)
ax[0].imshow(rescale_image(img_masked), cmap="gray")
ax[0].axis("off")
ax[1].imshow(rescale_image(img_masked), cmap="gray")
ax[1].imshow(nuc_masked, alpha=0.5)
ax[1].axis("off")
plt.tight_layout()
fig.subplots_adjust(wspace=0, hspace=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …